In [21]:
from importlib import reload
import numpy as np
from numpy import transpose, trace, multiply, power, dot
from numpy.linalg import multi_dot, matrix_power, norm
import scipy.stats as ss
from scipy.special import comb
from sklearn.cluster import KMeans
import math
import pandas as pd
import itertools
import time
from tqdm import tqdm
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import pickle
from TracyWidom import TracyWidom
import data_gen as dg
import stat_test as st
import visualizations as viz

In [22]:
# Reload modules in case of modifications
reload(dg)
reload(st)
reload(viz)

<module 'visualizations' from '/Users/louiscam/Dropbox (MIT)/SBMtesting/Simulations-Python/visualizations.py'>

# Compare empirical SNR of PET and Lei (2016)

In [23]:
# Simulation parameters
N_rep = 100
level = 0.05

### Null setting

In [24]:
# Model parameters
n = 500
K = 1
a = 0.2

In [25]:
# Generate data from SBM
np.random.seed(13)
data = []
for _ in tqdm(range(N_rep)):
    adj = dg.sample_null_adj_mat(n, a)
    data.append(adj)
data = np.array(data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 251.70it/s]


In [26]:
# Run PET on data
stats_PET = []
for c in tqdm(range(N_rep)): 
    A = data[c,:,:]
    stats_PET.append(st.PET_test(A, level)['test_stat'])
null_mean_PET = np.mean(stats_PET)
null_sd_PET = np.std(stats_PET)

# Run test of Lei (2016) on data
K0 = 1
stats_lei = []
for c in tqdm(range(N_rep)): 
    A = data[c,:,:]
    stats_lei.append(st.lei_test(A, K0, level)['test_stat'])
null_mean_lei = np.mean(stats_lei)
null_sd_lei = np.std(stats_lei)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:24<00:00,  4.16it/s]


### Symmetric alternative setting

In [27]:
# Model parameters
n = 500
K = 2
a = 0.2
b = 0.05

In [28]:
# Generate data from SBM
np.random.seed(13)
data = []
for _ in tqdm(range(N_rep)):
    adj = dg.sample_alt_adj_matrix_sym(n, K, a, b, exact=True)
    data.append(adj)
data = np.array(data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 152.80it/s]


In [29]:
# Run PET on data
stats_PET = []
for c in tqdm(range(N_rep)): 
    A = data[c,:,:]
    stats_PET.append(st.PET_test(A, level)['test_stat'])
alt_sym_mean_PET = np.mean(stats_PET)
alt_sym_sd_PET = np.std(stats_PET)

# Run test of Lei (2016) on data
K0 = 1
stats_lei = []
for c in tqdm(range(N_rep)): 
    A = data[c,:,:]
    stats_lei.append(st.lei_test(A, K0, level)['test_stat'])
alt_sym_mean_lei = np.mean(stats_lei)
alt_sym_sd_lei = np.std(stats_lei)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.41it/s]


### Asymmetric alternative setting

In [14]:
# Model parameters
n = 500
K = 2
b = 1
c = 1 
a = b+1/n**(1/2)
eta = np.concatenate((np.repeat(a,int(K/2)),np.repeat(b,int(K/2)))).reshape((K,1))
eta = eta/norm(eta)

In [15]:
# Generate data from SBM
np.random.seed(13)
data = []
for _ in tqdm(range(N_rep)):
    adj = dg.sample_alt_adj_matrix_rank1(n, K, eta, c, exact=True)
    data.append(adj)
data = np.array(data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 115.37it/s]


In [17]:
# Run PET on data
stats_PET = []
for c in tqdm(range(N_rep)): 
    A = data[c,:,:]
    stats_PET.append(st.PET_test(A, level)['test_stat'])
alt_asym_mean_PET = np.mean(stats_PET)
alt_asym_sd_PET = np.std(stats_PET)

# Run test of Lei (2016) on data
K0 = 1
stats_lei = []
for c in tqdm(range(N_rep)): 
    A = data[c,:,:]
    stats_lei.append(st.lei_test(A, K0, level)['test_stat'])
alt_asym_mean_lei = np.mean(stats_lei)
alt_asym_sd_lei = np.std(stats_lei)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.45it/s]


### Compute SNR

In [30]:
# Empirical SNR
snr_PET_sym = np.abs(alt_sym_mean_PET-null_mean_PET)/null_sd_PET
snr_lei_sym = np.abs(alt_sym_mean_lei-null_mean_lei)/null_sd_lei
snr_PET_asym = np.abs(alt_asym_mean_PET-null_mean_PET)/null_sd_PET
snr_lei_asym = np.abs(alt_asym_mean_lei-null_mean_lei)/null_sd_lei
print(f'Empirical SNR of PET (symmetric alternative) = {snr_PET_sym}')
print(f'Empirical SNR of Lei (symmetric alternative) = {snr_lei_sym}')
print(f'Empirical SNR of PET (asymmetric alternative) = {snr_PET_asym}')
print(f'Empirical SNR of Lei (asymmetric alternative) = {snr_lei_asym}')

Empirical SNR of PET (symmetric alternative) = 30678.93387127697
Empirical SNR of Lei (symmetric alternative) = 174.33769823570998
Empirical SNR of PET (asymmetric alternative) = 7.018241859310978
Empirical SNR of Lei (asymmetric alternative) = 0.1682732702869609
